In [1]:
import pandas as pd
import json

In [2]:
with open('../data/squad_data/dev-v2.0.json', 'r') as squad_file:
    squad_dev = json.load(squad_file)

In [3]:
all_answers = []
for text in squad_dev['data']:
    for para in text['paragraphs']:
        all_answers.append(pd.DataFrame.from_dict(para['qas'], orient='columns'))
answers_df = pd.concat(all_answers)

In [4]:
all_answers = []
for answers in list(answers_df['answers']):
    this_answers = []
    for a in answers:
        this_answers.append(a['text'])
    all_answers.append(this_answers)
answers_df['answers_lists'] = all_answers
answers_df = answers_df.set_index('id')
answers_df['is_impossible'] = answers_df['is_impossible'].astype(bool)

In [14]:
with open('../models/bert-prune-40-squad/predictions.json', 'r') as preds_file:
    predictions_json = json.load(preds_file)
    
predictions_df = pd.DataFrame.from_dict(predictions_json, orient='index')
ans_pred_df = answers_df.join(predictions_df, how='left')

In [16]:
# all entries with is_impossible = False
possible_answers_df = ans_pred_df[~ans_pred_df['is_impossible']]
# Check if the prediction is in the list of possible answers
possible_answers_df['correct'] = possible_answers_df.apply(lambda x: x[0] in x.answers_lists, axis=1)

results = possible_answers_df.correct.value_counts()
results[0]/(results[0]+results[1])

/tmp/ipykernel_125067/2031579089.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  possible_answers_df['correct'] = possible_answers_df.apply(lambda x: x[0] in x.answers_lists, axis=1)


0.599527665317139

In [155]:
ans_pred_df['correct'] = ans_pred_df.apply(lambda x: x[0] in x.answers_lists, axis=1)
# Elementwise logical AND
ans_pred_df['possible_correct'] = ans_pred_df['is_impossible'].astype(bool) & (ans_pred_df[0] == '').astype(bool)

In [158]:
ans_pred_df['all_correct'] = ans_pred_df['correct'] | ans_pred_df['possible_correct']

In [160]:
all_results = ans_pred_df['all_correct'].value_counts()

In [161]:
all_results[0]/(all_results[0]+all_results[1])

0.6211572475364272